# Chapter 1: GPU Architecture & Fundamentals

## 🎯 Learning Objectives

By the end of this chapter, you will:
- **Understand modern GPU architecture** from the ground up
- **Master memory hierarchy** and its impact on ML performance
- **Analyze CUDA execution models** and their implications
- **Optimize memory access patterns** for maximum bandwidth utilization
- **Leverage Tensor Cores** for mixed-precision acceleration

---

## 🧠 Why GPU Architecture Matters for LLMs

### **The Performance Reality**
Modern LLMs like GPT-4 and Claude require:
- **Massive parallelism**: Billions of parameters processed simultaneously
- **High memory bandwidth**: Moving TB/s of data between compute units
- **Specialized compute**: Tensor operations optimized for AI workloads
- **Efficient communication**: Multi-GPU coordination for large models

**Understanding GPU architecture isn't optional—it's the foundation of all LLM optimization.**

---

## 🏗️ Modern GPU Architecture Deep Dive

### **NVIDIA GPU Hierarchy (2024)**

```
Consumer/Research: RTX 4090, RTX 4080
                   ↓
Professional:      A6000, RTX 6000 Ada
                   ↓
Data Center:       T4, V100, A100, H100
                   ↓
Next-Gen:          B100, GB200 (2024-2025)
```

### **Key Architectural Components**

#### 1. **Streaming Multiprocessors (SMs)**
- **Definition**: The basic computational unit containing multiple cores
- **T4**: 40 SMs × 64 CUDA cores = 2,560 total cores
- **A100**: 108 SMs × 64 CUDA cores = 6,912 total cores
- **H100**: 114 SMs × 128 CUDA cores = 14,592 total cores

#### 2. **CUDA Cores**
- **Purpose**: Execute single-precision floating-point operations
- **Capability**: One FP32 operation per clock cycle
- **Limitation**: Not optimized for AI workloads (hence Tensor Cores)

#### 3. **Tensor Cores**
- **Revolution**: Specialized AI acceleration units
- **T4**: 320 Tensor Cores (1st gen) - 65 TFLOPS FP16
- **A100**: 432 Tensor Cores (3rd gen) - 312 TFLOPS FP16
- **H100**: 456 Tensor Cores (4th gen) - 1,979 TFLOPS FP8

#### 4. **RT Cores** (Graphics-focused, less relevant for LLMs)
- Ray tracing acceleration
- Some applications in 3D AI and NeRF training

---

## 🧮 Memory Hierarchy: The Performance Bottleneck

### **The Memory Wall Problem**

**Compute performance** has grown exponentially:
- 2016: P100 - 21 TFLOPS FP16
- 2024: H100 - 1,979 TFLOPS FP8 (94x improvement)

**Memory bandwidth** has grown linearly:
- 2016: P100 - 732 GB/s
- 2024: H100 - 3,350 GB/s (4.6x improvement)

**Result**: Most LLM operations are memory-bound, not compute-bound!

### **GPU Memory Hierarchy (Fastest to Slowest)**

#### 1. **Registers**
- **Location**: Inside each CUDA core
- **Size**: 256KB per SM (T4), 512KB per SM (A100)
- **Bandwidth**: ~20 TB/s
- **Latency**: 0 cycles
- **Use Case**: Thread-local variables, loop counters

#### 2. **Shared Memory**
- **Location**: Shared within each SM
- **Size**: 64KB per SM (configurable with L1 cache)
- **Bandwidth**: ~15 TB/s
- **Latency**: 1-2 cycles
- **Use Case**: Block-level data sharing, manual caching

#### 3. **L1 Cache**
- **Location**: Per SM (shared with shared memory)
- **Size**: 128KB per SM (configurable split)
- **Bandwidth**: ~1 TB/s
- **Latency**: 20-40 cycles
- **Use Case**: Automatic caching of global memory accesses

#### 4. **L2 Cache**
- **Location**: Shared across all SMs
- **Size**: 5MB (T4), 40MB (A100), 50MB (H100)
- **Bandwidth**: ~500 GB/s
- **Latency**: 200-400 cycles
- **Use Case**: Cross-SM data sharing, texture caching

#### 5. **Global Memory (HBM)**
- **Location**: High Bandwidth Memory stacks
- **Size**: 16GB (T4), 80GB (A100), 80GB (H100)
- **Bandwidth**: 320GB/s (T4), 1,935GB/s (A100), 3,350GB/s (H100)
- **Latency**: 400-800 cycles
- **Use Case**: Model parameters, activations, gradients

#### 6. **System Memory**
- **Location**: CPU DRAM via PCIe
- **Size**: System dependent (32GB-1TB+)
- **Bandwidth**: ~50 GB/s (PCIe 4.0 x16)
- **Latency**: 1000+ cycles
- **Use Case**: Data staging, checkpointing

---

## 🔬 CUDA Execution Model

### **Thread Hierarchy**

```
Grid (entire kernel launch)
├── Block 0 (executed on one SM)
│   ├── Warp 0 (32 threads, SIMD execution)
│   │   ├── Thread 0
│   │   ├── Thread 1
│   │   └── ... Thread 31
│   ├── Warp 1 (32 threads)
│   └── ...
├── Block 1
└── ...
```

### **Key Concepts**

#### **Warp Execution**
- **32 threads** execute the same instruction simultaneously (SIMD)
- **Divergence penalty**: If threads take different branches, performance drops
- **Memory coalescing**: Adjacent threads should access adjacent memory

#### **Occupancy**
- **Definition**: Ratio of active warps to maximum possible warps per SM
- **Factors**: Register usage, shared memory usage, block size
- **Target**: 75-100% occupancy for memory-bound kernels

---

## 🧬 Tensor Cores: The AI Revolution

### **Evolution of Tensor Cores**

#### **1st Generation (V100)**
- **Mixed Precision**: FP16 inputs, FP32 accumulate
- **Matrix Size**: 4×4×4 (A×B+C)
- **Performance**: 125 TFLOPS peak

#### **2nd Generation (T4, RTX series)**
- **Added INT8/INT4** support
- **Improved sparsity** handling
- **Better compiler** integration

#### **3rd Generation (A100)**
- **BF16 support** (better numerical range than FP16)
- **TF32 mode** (FP32 range, FP16 precision) - 156 TFLOPS
- **Sparsity optimization** (2:4 structured sparsity)

#### **4th Generation (H100)**
- **FP8 support** (E4M3 and E5M2 formats)
- **Transformer Engine** integration
- **1,979 TFLOPS FP8** peak performance

### **Why Tensor Cores Matter for LLMs**

#### **Traditional GEMM (General Matrix Multiply)**
```python
# CUDA Cores: Element-wise operations
for i in range(M):
    for j in range(N):
        C[i][j] = 0
        for k in range(K):
            C[i][j] += A[i][k] * B[k][j]  # One FP32 op per cycle
```

#### **Tensor Core GEMM**
```python
# Tensor Cores: Fused matrix operations
# Single instruction computes 4x4 matrix multiply-accumulate
C_4x4 = A_4x4 @ B_4x4 + C_4x4  # 64 FP16 ops in one cycle!
```

**Performance Gain**: 16-32x speedup for mixed-precision workloads

---

## 📊 LLM-Specific Performance Considerations

### **Transformer Architecture Impact**

#### **Attention Mechanism**
- **GEMM operations**: Q×K^T, Softmax×V
- **Memory pattern**: Non-sequential access for attention weights
- **Optimization**: FlashAttention, PagedAttention

#### **Feed-Forward Networks**
- **Large GEMMs**: Hidden → 4×Hidden → Hidden
- **Memory pattern**: Sequential, tensor-core friendly
- **Optimization**: Activation checkpointing, mixed precision

#### **Layer Normalization**
- **Reduction operations**: Mean, variance across hidden dimension
- **Memory bound**: Low arithmetic intensity
- **Optimization**: Kernel fusion, FP16 computation

### **Sequence Length Scaling**

#### **Memory Complexity**
- **Attention**: O(n²) for sequence length n
- **Feed-forward**: O(n) linear scaling
- **Total model**: Dominated by attention for long sequences

#### **Compute Complexity**
- **Training**: O(n²) attention + O(n) feed-forward
- **Inference**: O(n) per token (with KV caching)

---

## 🔧 Practical Architecture Analysis

Let's analyze your current GPU and understand its capabilities:

In [ ]:
import torch
import subprocess
import json
from typing import Dict, Any

def analyze_gpu_architecture() -> Dict[str, Any]:
    """
    Comprehensive GPU architecture analysis
    
    This function demonstrates how to programmatically analyze
    GPU capabilities for LLM optimization decisions.
    """
    
    if not torch.cuda.is_available():
        return {"error": "No CUDA GPU detected"}
    
    # Basic PyTorch GPU info
    device_id = 0
    props = torch.cuda.get_device_properties(device_id)
    
    analysis = {
        "basic_info": {
            "name": props.name,
            "compute_capability": f"{props.major}.{props.minor}",
            "total_memory_gb": props.total_memory / (1024**3),
            "multiprocessor_count": props.multiprocessor_count,
        }
    }
    
    # Derive architecture details from compute capability
    major, minor = props.major, props.minor
    
    if major == 7 and minor == 5:  # T4
        analysis["architecture"] = {
            "generation": "Turing",
            "year": "2018",
            "cuda_cores_per_sm": 64,
            "total_cuda_cores": props.multiprocessor_count * 64,
            "tensor_cores": "2nd Gen (320 cores)",
            "tensor_core_performance": "65 TFLOPS FP16",
            "memory_type": "GDDR6",
            "memory_bandwidth_gbs": 320,
            "l2_cache_mb": 5.0,
            "shared_memory_per_sm_kb": 64,
        }
    elif major == 8 and minor == 0:  # A100
        analysis["architecture"] = {
            "generation": "Ampere",
            "year": "2020",
            "cuda_cores_per_sm": 64,
            "total_cuda_cores": props.multiprocessor_count * 64,
            "tensor_cores": "3rd Gen (432 cores)",
            "tensor_core_performance": "312 TFLOPS FP16, 156 TFLOPS TF32",
            "memory_type": "HBM2e",
            "memory_bandwidth_gbs": 1935,
            "l2_cache_mb": 40.0,
            "shared_memory_per_sm_kb": 164,
        }
    elif major == 9 and minor == 0:  # H100
        analysis["architecture"] = {
            "generation": "Hopper",
            "year": "2022",
            "cuda_cores_per_sm": 128,
            "total_cuda_cores": props.multiprocessor_count * 128,
            "tensor_cores": "4th Gen (456 cores)",
            "tensor_core_performance": "1979 TFLOPS FP8, 989 TFLOPS FP16",
            "memory_type": "HBM3",
            "memory_bandwidth_gbs": 3350,
            "l2_cache_mb": 50.0,
            "shared_memory_per_sm_kb": 228,
        }
    else:
        analysis["architecture"] = {
            "generation": "Unknown",
            "note": f"Compute capability {major}.{minor} not in database"
        }
    
    # Calculate derived metrics
    if "architecture" in analysis and "memory_bandwidth_gbs" in analysis["architecture"]:
        bandwidth = analysis["architecture"]["memory_bandwidth_gbs"]
        memory_gb = analysis["basic_info"]["total_memory_gb"]
        
        analysis["performance_analysis"] = {
            "memory_bandwidth_per_gb": bandwidth / memory_gb,
            "bandwidth_utilization_threshold": "80%+ for memory-bound kernels",
            "recommended_batch_size_fp16": f"~{int(memory_gb * 0.8 / 2)} for inference",
            "tensor_core_recommendation": "Use mixed precision (FP16) for maximum performance"
        }
    
    # Try to get additional info from nvidia-smi
    try:
        result = subprocess.run([
            'nvidia-smi', '--query-gpu=driver_version,cuda_version,pci.bus_id,pcie.link.gen.current,pcie.link.width.current',
            '--format=csv,noheader,nounits'
        ], capture_output=True, text=True, timeout=10)
        
        if result.returncode == 0:
            values = result.stdout.strip().split(', ')
            analysis["system_info"] = {
                "driver_version": values[0],
                "cuda_version": values[1],
                "pci_bus_id": values[2],
                "pcie_generation": values[3],
                "pcie_width": values[4] + "x"
            }
    except:
        analysis["system_info"] = {"note": "nvidia-smi not available"}
    
    return analysis

# Analyze current GPU
print("🔍 Analyzing GPU Architecture...")
gpu_analysis = analyze_gpu_architecture()

# Pretty print results
import pprint
pp = pprint.PrettyPrinter(indent=2, width=80)
pp.pprint(gpu_analysis)

## 📈 Memory Bandwidth Utilization Test

Let's measure actual memory bandwidth and compare it to theoretical limits:

In [ ]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

def benchmark_memory_bandwidth(device='cuda'):
    """
    Measure actual memory bandwidth using various access patterns
    
    Educational Purpose:
    This benchmark demonstrates how different memory access patterns
    affect bandwidth utilization - a key concept for LLM optimization.
    """
    
    if device == 'cuda' and not torch.cuda.is_available():
        print("CUDA not available, using CPU")
        device = 'cpu'
    
    results = {}
    
    # Test different data sizes (MB)
    sizes_mb = [1, 4, 16, 64, 256, 1024]
    
    for size_mb in sizes_mb:
        if device == 'cuda':
            # Check if size fits in GPU memory
            available_mb = torch.cuda.get_device_properties(0).total_memory / (1024**2)
            if size_mb * 3 > available_mb * 0.8:  # Need 3x size for safety
                continue
        
        size_elements = (size_mb * 1024 * 1024) // 4  # FP32 elements
        
        # Create test tensors
        a = torch.randn(size_elements, device=device, dtype=torch.float32)
        b = torch.randn(size_elements, device=device, dtype=torch.float32)
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        # Warm up
        for _ in range(5):
            c = a + b
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        # Benchmark vector addition (memory bound operation)
        num_iterations = max(10, 1000 // size_mb)
        
        start_time = time.time()
        for _ in range(num_iterations):
            c = a + b  # Read A, Read B, Write C = 3x data movement
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        end_time = time.time()
        
        # Calculate bandwidth
        total_time = end_time - start_time
        bytes_transferred = size_mb * 1024 * 1024 * 3 * num_iterations  # 3x for read A, read B, write C
        bandwidth_gbs = (bytes_transferred / (1024**3)) / total_time
        
        results[size_mb] = {
            'bandwidth_gbs': bandwidth_gbs,
            'iterations': num_iterations,
            'time_seconds': total_time
        }
        
        print(f"Size: {size_mb:4d} MB, Bandwidth: {bandwidth_gbs:6.1f} GB/s")
        
        # Clean up
        del a, b, c
        if device == 'cuda':
            torch.cuda.empty_cache()
    
    return results

print("📊 Memory Bandwidth Benchmark")
print("Testing vector addition (memory-bound operation)...\n")

bandwidth_results = benchmark_memory_bandwidth()

if bandwidth_results:
    # Plot results
    sizes = list(bandwidth_results.keys())
    bandwidths = [bandwidth_results[s]['bandwidth_gbs'] for s in sizes]
    
    plt.figure(figsize=(10, 6))
    plt.semilogx(sizes, bandwidths, 'bo-', linewidth=2, markersize=8)
    plt.xlabel('Data Size (MB)')
    plt.ylabel('Bandwidth (GB/s)')
    plt.title('Memory Bandwidth vs Data Size')
    plt.grid(True, alpha=0.3)
    
    # Add theoretical bandwidth line if we know the GPU
    if torch.cuda.is_available():
        props = torch.cuda.get_device_properties(0)
        if "T4" in props.name:
            plt.axhline(y=320, color='r', linestyle='--', alpha=0.7, label='T4 Theoretical (320 GB/s)')
            plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    max_bandwidth = max(bandwidths)
    print(f"\n🎯 Peak measured bandwidth: {max_bandwidth:.1f} GB/s")
    
    if torch.cuda.is_available():
        props = torch.cuda.get_device_properties(0)
        if "T4" in props.name:
            efficiency = (max_bandwidth / 320) * 100
            print(f"📈 Bandwidth efficiency: {efficiency:.1f}% of T4 theoretical peak")
            
            if efficiency > 80:
                print("✅ Excellent bandwidth utilization!")
            elif efficiency > 60:
                print("🟡 Good bandwidth utilization")
            else:
                print("🔴 Low bandwidth utilization - check for bottlenecks")
else:
    print("❌ Benchmark failed - insufficient memory or other issues")

## 🎯 Key Takeaways

### **Memory is King**
- Modern LLMs are **memory-bound**, not compute-bound
- **Bandwidth utilization** is more important than peak FLOPS
- **Memory access patterns** determine performance

### **Tensor Cores are Essential**
- **16-32x speedup** for mixed-precision operations
- **Mixed precision training** is not optional for production
- **Proper data layout** is required for tensor core utilization

### **Architecture Matters**
- **Different GPUs** have different optimization strategies
- **Understanding your hardware** guides optimization decisions
- **Theoretical limits** set expectations for achievable performance

---

## 📚 Additional Resources

### **NVIDIA Documentation**
- [CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)
- [Tensor Core Programming](https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/)
- [GPU Architecture Whitepapers](https://www.nvidia.com/en-us/data-center/resources/)

### **Academic Papers**
- "Analyzing Deep Neural Networks with Tensor Cores" (ISCA 2020)
- "Understanding the Memory and Compute Requirements of LLMs" (MLSys 2023)
- "FlashAttention: Fast and Memory-Efficient Exact Attention" (NeurIPS 2022)

### **Industry Benchmarks**
- [MLPerf Training and Inference](https://mlperf.org/)
- [NVIDIA Deep Learning Examples](https://github.com/NVIDIA/DeepLearningExamples)

---

**Next: Chapter 2 - Scientific Profiling Methodology** 🔬

*In the next chapter, we'll learn how to systematically identify and measure performance bottlenecks using scientific methodology and production-grade tools.*